<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/langgraphy_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Langchain Memory Types with Examples

Welcome! This notebook is designed to guide you through understanding and implementing several fundamental **Langchain Memory Types**.

**What is Memory in Langchain?**
Large Language Models (LLMs) are inherently stateless, meaning each incoming query is processed independently without knowledge of past interactions. Memory provides LLMs with a way to retain information about previous interactions in a conversation. This allows for more coherent, context-aware, and engaging conversations or task executions over multiple turns.

**Why are Memory Types Important?**
Different applications have different memory requirements. Some might need to remember the entire conversation, while others might only need a summary or a recent window of interactions due to token limits or specific task needs. Langchain offers various memory types to cater to these diverse scenarios.

**In this notebook, we will explore and implement:**
1.  **ConversationBufferMemory:** Remembers the full conversation history.
2.  **ConversationBufferWindowMemory:** Remembers a fixed number of recent interactions.
3.  **ConversationSummaryMemory:** Creates and remembers a summary of the conversation.

Let's dive in!

## Table of Contents
1. [Setup](#setup-memory)
    - [Step 1: Install Necessary Libraries](#install-libraries-memory)
    - [Step 2: Configure OpenAI API Key](#configure-api-key-memory)
    - [Step 3: Common Imports and LLM Initialization](#common-imports-memory)
2. [Memory Type 1: ConversationBufferMemory](#memory-buffer)
    - [Concept](#concept-buffer)
    - [Use Case Example](#usecase-buffer)
    - [1.1 Demo: ConversationBufferMemory](#demo-buffer)
3. [Memory Type 2: ConversationBufferWindowMemory](#memory-buffer-window)
    - [Concept](#concept-buffer-window)
    - [Use Case Example](#usecase-buffer-window)
    - [2.1 Demo: ConversationBufferWindowMemory](#demo-buffer-window)
4. [Memory Type 3: ConversationSummaryMemory](#memory-summary)
    - [Concept](#concept-summary)
    - [Use Case Example](#usecase-summary)
    - [3.1 Demo: ConversationSummaryMemory](#demo-summary)
5. [Conclusion](#conclusion-memory)

## Setup <a name="setup-memory"></a>

Before we can explore different memory types, we need to set up our environment.

### Step 1: Install Necessary Libraries <a name="install-libraries-memory"></a>

We'll need a few key Python libraries:
-   `langchain`: The core library for building applications with LLMs.
-   `langchain-openai`: Specifically for interacting with OpenAI's models.
-   `python-dotenv` (optional): Useful for managing API keys if you store them in a `.env` file (not strictly used in this notebook's interactive key input).

In [ ]:
!pip install -q langchain langchain-openai python-dotenv

### Step 2: Configure OpenAI API Key <a name="configure-api-key-memory"></a>

To use OpenAI's models, you need an API key.
1.  If you don't have one, sign up at [platform.openai.com](https://platform.openai.com/).
2.  The code below will set your API key as an environment variable.

**Security Note:** The example below includes a placeholder API key for demonstration purposes. In a real-world scenario, **never hardcode your API keys directly into your code, especially if you plan to share it or commit it to version control.** Use environment variables or a secrets management system. The `getpass` method is a more secure way to input it interactively if you don't have it set as an environment variable already.

In [ ]:
import os
import getpass

# **WARNING:** This is NOT recommended for production or shared code.
# Replace with your actual key or use the getpass method below.
hardcoded_api_key = ""
os.environ["OPENAI_API_KEY"] = hardcoded_api_key
print("OpenAI API Key has been set (using the placeholder from the script - REPLACE IT).")

# # --- More Secure Method (Recommended if key is not already an environment variable) ---
# if "OPENAI_API_KEY" not in os.environ:
#     print("OpenAI API Key not found in environment variables.")
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
#     print("OpenAI API Key set for this session.")
# elif not os.environ["OPENAI_API_KEY"].startswith("sk-proj-"):
#     print("API key in environment variable might be invalid or a placeholder.")
#     print("If you have a new key, consider updating it or inputting manually.")
#     use_manual_input = input("Input API key manually? (yes/no): ").lower()
#     if use_manual_input == 'yes':
#        os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
#        print("OpenAI API Key updated for this session.")
#     else:
#        print("Using existing (potentially placeholder) API key from environment variable.")
# else:
#     print("OpenAI API Key found in environment variables.")

# Final check if the key is still the placeholder
if os.environ.get("OPENAI_API_KEY") == "sk-proj-YOUR_OPENAI_API_KEY_HERE_REPLACE_ME":
    print("\nWARNING: You are using a placeholder API key.")
    print("The LLM calls will likely fail. Please replace it with your actual OpenAI API key.")

OpenAI API Key has been set (using the placeholder from the script - REPLACE IT).


### Step 3: Common Imports and LLM Initialization <a name="common-imports-memory"></a>

Let's import the necessary modules from LangChain and initialize our LLM. We'll use OpenAI's `gpt-4.1` model for these examples. The `temperature` parameter controls the randomness of the LLM's output; a lower value (like 0.7) makes the output more focused but still allows for some creativity.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1", temperature=0.7)
print("OpenAI LLM (gpt-4.1) initialized.")

OpenAI LLM (gpt-4.1) initialized.


## Memory Type 1: ConversationBufferMemory <a name="memory-buffer"></a>

### Concept <a name="concept-buffer"></a>
**ConversationBufferMemory** is one of the simplest memory types. It stores the entire conversation history as a sequence of messages in a buffer. When the chain is called, all messages in the buffer are passed to the LLM as context.

This is useful for maintaining a complete record of the interaction, allowing the LLM to refer back to any part of the conversation. However, for very long conversations, this can lead to exceeding the LLM's context window limit and can become expensive in terms of token usage.

### Use Case Example <a name="usecase-buffer"></a>
- **Short to Medium Length Chatbots:** Ideal for chatbots where remembering the full context of the current session is crucial for coherent responses, such as customer service interactions or interactive storytelling.
- **Task-Oriented Dialogues:** When a user is trying to achieve a specific goal that might involve multiple steps, remembering all previous inputs and outputs is important (e.g., booking a flight, configuring a product).

### 1.1 Demo: ConversationBufferMemory <a name="demo-buffer"></a>
Let's see how `ConversationBufferMemory` works in practice.

In [ ]:
print("--- DEMO: ConversationBufferMemory ---")

# Initialize memory
buffer_memory = ConversationBufferMemory()

# Initialize ConversationChain with the LLM and buffer_memory
conversation_with_buffer = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    verbose=False # Set to True to see the full prompt being sent to LLM
)

print("Initialized ConversationChain with ConversationBufferMemory.")

# First interaction
user_input_1 = "Hi there! I'm planning a trip to Italy."
print(f"\nUser: {user_input_1}")
ai_response_1 = conversation_with_buffer.predict(input=user_input_1)
print(f"AI: {ai_response_1}")

# Second interaction
user_input_2 = "What are some must-see cities?"
print(f"\nUser: {user_input_2}")
ai_response_2 = conversation_with_buffer.predict(input=user_input_2)
print(f"AI: {ai_response_2}")

# Third interaction - LLM should remember the context (Italy)
user_input_3 = "And what about food recommendations for those places?"
print(f"\nUser: {user_input_3}")
ai_response_3 = conversation_with_buffer.predict(input=user_input_3)
print(f"AI: {ai_response_3}")

print("\n--- Current Conversation Buffer ---")
print(buffer_memory.buffer)
print("--- End of ConversationBufferMemory Demo ---")

--- DEMO: ConversationBufferMemory ---
Initialized ConversationChain with ConversationBufferMemory.

User: Hi there! I'm planning a trip to Italy.
AI: That sounds amazing! Italy is such a beautiful country with so much to offer—gorgeous cities, incredible food, stunning landscapes, and a rich history. Do you already know which regions or cities you want to visit? For example, are you thinking about the classic trio of Rome, Florence, and Venice, or are you interested in exploring less-touristy places like Puglia, Umbria, or Sicily? If you give me an idea of your interests—art, history, food, beaches, hiking, shopping—I can suggest some tailored recommendations and tips for your trip!

User: What are some must-see cities?
AI: That sounds amazing! Italy is such a beautiful country with so much to offer—gorgeous cities, incredible food, stunning landscapes, and a rich history. Do you already know which regions or cities you want to visit? For example, are you thinking about the classic tr

## Memory Type 2: ConversationBufferWindowMemory <a name="memory-buffer-window"></a>

### Concept <a name="concept-buffer-window"></a>
**ConversationBufferWindowMemory** is similar to `ConversationBufferMemory` but with a crucial difference: it only keeps a specified number (`k`) of the most recent interactions in its buffer.

This is useful for preventing the conversation history from growing too large, which helps manage token limits and costs. It ensures that the LLM has access to the immediate past context, but older parts of the conversation are eventually forgotten.

### Use Case Example <a name="usecase-buffer-window"></a>
- **Long-Running Chatbots:** For chatbots that might have extended interactions, this prevents the context from becoming too large and unwieldy. The bot will primarily focus on recent turns.
- **Resource-Constrained Environments:** When token count is a significant concern (e.g., due to cost or model limitations).
- **Applications where only recent context matters:** For example, a troubleshooting bot might only need the last few steps taken by the user, not the entire history from days ago.

### 2.1 Demo: ConversationBufferWindowMemory <a name="demo-buffer-window"></a>
Let's see `ConversationBufferWindowMemory` in action, with `k=2` (meaning it will remember the last 2 pairs of human/AI messages).

In [ ]:
print("--- DEMO: ConversationBufferWindowMemory (k=2) ---")

# Initialize memory with k=2 (remembers last 2 interactions)
# An interaction is one human message and one AI message.
buffer_window_memory = ConversationBufferWindowMemory(k=2)

# Initialize ConversationChain
conversation_with_window = ConversationChain(
    llm=llm,
    memory=buffer_window_memory,
    verbose=False
)
print("Initialized ConversationChain with ConversationBufferWindowMemory (k=2).")

# Interaction 1
user_input_w1 = "My favorite color is blue."
print(f"\nUser: {user_input_w1}")
ai_response_w1 = conversation_with_window.predict(input=user_input_w1)
print(f"AI: {ai_response_w1}")
print(f"Current Buffer Window (after 1st interaction):\n{buffer_window_memory.buffer}\n")

# Interaction 2
user_input_w2 = "My favorite animal is a dog."
print(f"\nUser: {user_input_w2}")
ai_response_w2 = conversation_with_window.predict(input=user_input_w2)
print(f"AI: {ai_response_w2}")
print(f"Current Buffer Window (after 2nd interaction):\n{buffer_window_memory.buffer}\n")

# Interaction 3
user_input_w3 = "I love to play chess."
print(f"\nUser: {user_input_w3}")
ai_response_w3 = conversation_with_window.predict(input=user_input_w3)
print(f"AI: {ai_response_w3}")
print(f"Current Buffer Window (after 3rd interaction - 'blue' should be gone):\n{buffer_window_memory.buffer}\n")

# Interaction 4 - Ask about favorite color. LLM might not remember 'blue'.
user_input_w4 = "What is my favorite color?"
print(f"\nUser: {user_input_w4}")
ai_response_w4 = conversation_with_window.predict(input=user_input_w4)
print(f"AI: {ai_response_w4}") # The AI might not know 'blue' anymore.

# Interaction 5 - Ask about favorite animal. LLM should remember 'dog'.
user_input_w5 = "What is my favorite animal?"
print(f"\nUser: {user_input_w5}")
ai_response_w5 = conversation_with_window.predict(input=user_input_w5)
print(f"AI: {ai_response_w5}") # The AI should still remember 'dog'.

print("\n--- Final Conversation Buffer Window ---")
print(buffer_window_memory.buffer)
print("--- End of ConversationBufferWindowMemory Demo ---")

--- DEMO: ConversationBufferWindowMemory (k=2) ---
Initialized ConversationChain with ConversationBufferWindowMemory (k=2).

User: My favorite color is blue.


C:\Users\prajwalrao\AppData\Local\Temp\ipykernel_41388\466045682.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_window_memory = ConversationBufferWindowMemory(k=2)


AI: Blue is a wonderful choice! It’s often associated with calmness, trust, and creativity. There are so many beautiful shades, from the deep navy of a night sky to the bright turquoise of tropical waters. Do you have a favorite shade of blue, or is it more about the general vibe of the color for you?
Current Buffer Window (after 1st interaction):
Human: My favorite color is blue.
AI: Blue is a wonderful choice! It’s often associated with calmness, trust, and creativity. There are so many beautiful shades, from the deep navy of a night sky to the bright turquoise of tropical waters. Do you have a favorite shade of blue, or is it more about the general vibe of the color for you?


User: My favorite animal is a dog.
AI: Dogs are amazing! They’re loyal, loving, and come in so many different breeds, each with their own unique personalities. Whether it’s a playful Labrador, a clever Border Collie, or a tiny, spirited Chihuahua, dogs have been human companions for thousands of years. Do you 

## Memory Type 3: ConversationSummaryMemory <a name="memory-summary"></a>

### Concept <a name="concept-summary"></a>
**ConversationSummaryMemory** creates a summary of the conversation over time. As the conversation progresses, it condenses the previous interactions into a concise summary that is then passed to the LLM as context along with the latest user input.

This is particularly useful for long conversations where keeping the entire history is impractical due to token limits. The summary aims to capture the key points and evolving context, allowing the LLM to maintain a sense of the overall discussion without needing every detail.

### Use Case Example <a name="usecase-summary"></a>
- **Long-Term Personal Assistants:** An assistant that interacts with a user over days or weeks needs to remember key preferences, ongoing tasks, or important past decisions without storing every single message.
- **Complex Problem Solving:** When a conversation involves exploring multiple threads or a lengthy information gathering process, a summary helps the LLM keep track of the main objectives and findings.
- **Meeting Summarizers or Recappers:** While not its primary use as a chain memory, the underlying principle is similar to generating ongoing summaries of discussions.

### 3.1 Demo: ConversationSummaryMemory <a name="demo-summary"></a>
Let's observe how `ConversationSummaryMemory` generates and uses a summary.

In [ ]:
print("--- DEMO: ConversationSummaryMemory ---")

# Initialize summary memory. It needs an LLM to generate the summaries.
summary_memory = ConversationSummaryMemory(llm=llm)

# Initialize ConversationChain
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=False # Set to True to see the prompt including the summary
)
print("Initialized ConversationChain with ConversationSummaryMemory.")

# Interaction 1
user_input_s1 = "I want to learn about ancient civilizations. Let's start with Egypt."
print(f"\nUser: {user_input_s1}")
ai_response_s1 = conversation_with_summary.predict(input=user_input_s1)
print(f"AI: {ai_response_s1}")
print(f"Current Summary (after 1st interaction):\n{summary_memory.buffer}\n")

# Interaction 2
user_input_s2 = "Tell me about the pyramids and their significance."
print(f"\nUser: {user_input_s2}")
ai_response_s2 = conversation_with_summary.predict(input=user_input_s2)
print(f"AI: {ai_response_s2}")
print(f"Current Summary (after 2nd interaction):\n{summary_memory.buffer}\n")

# Interaction 3
user_input_s3 = "Now, let's switch to the Roman Empire. What were their major engineering achievements?"
print(f"\nUser: {user_input_s3}")
ai_response_s3 = conversation_with_summary.predict(input=user_input_s3)
print(f"AI: {ai_response_s3}")
print(f"Current Summary (after 3rd interaction - should reflect the topic change):\n{summary_memory.buffer}\n")

# Interaction 4 - Ask a question that relies on the summarized context.
user_input_s4 = "Comparing the two we discussed, which one had more lasting architectural influence?"
print(f"\nUser: {user_input_s4}")
ai_response_s4 = conversation_with_summary.predict(input=user_input_s4)
print(f"AI: {ai_response_s4}")

print("\n--- Final Conversation Summary ---")
print(summary_memory.buffer)
print("--- End of ConversationSummaryMemory Demo ---")

--- DEMO: ConversationSummaryMemory ---
Initialized ConversationChain with ConversationSummaryMemory.

User: I want to learn about ancient civilizations. Let's start with Egypt.


C:\Users\prajwalrao\AppData\Local\Temp\ipykernel_41388\3709819622.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm)


AI: Absolutely! Ancient Egypt is one of the most fascinating civilizations in world history, lasting for over 3,000 years—from around 3100 BCE to 30 BCE. It was centered along the Nile River, which provided fertile land for agriculture, making Egypt a powerhouse of the ancient world.

Some key aspects of ancient Egyptian civilization include:

1. **Pharaohs**: The rulers of Egypt were called pharaohs, and they were considered both political and religious leaders, often thought to be gods on earth. Famous pharaohs include Tutankhamun (King Tut), Ramses II, and Cleopatra VII.

2. **Pyramids and Monuments**: Egyptians are renowned for their monumental architecture, especially the pyramids. The most famous is the Great Pyramid of Giza, built for Pharaoh Khufu around 2560 BCE. They also built temples, obelisks, and the Sphinx.

3. **Religion and Afterlife**: Religion was central to Egyptian life. They believed in many gods, like Ra (the sun god), Osiris (god of the underworld), and Isis (go

## Conclusion <a name="conclusion-memory"></a>

Congratulations on exploring these Langchain memory types!

We've covered:
1.  **ConversationBufferMemory:** Retains the entire chat history, good for short to medium conversations where full context is key.
2.  **ConversationBufferWindowMemory:** Keeps a sliding window of the last `k` interactions, useful for managing token limits in longer conversations while maintaining recent context.
3.  **ConversationSummaryMemory:** Condenses the conversation into a running summary, excellent for very long interactions where only the gist of past exchanges is needed.

**Key Langchain Concepts Demonstrated:**
-   **Memory Initialization:** Creating instances of different memory classes.
-   **ConversationChain:** Integrating memory with an LLM to have context-aware conversations.
-   **Context Management:** Understanding how different memory types store and provide context to the LLM.

Choosing the right memory type is crucial for building effective and efficient Langchain applications. The best choice depends on the specific requirements of your task, the expected length of interaction, and token/cost considerations.

**Further Exploration:**
-   Explore other memory types like `ConversationKGMemory` (knowledge graph memory) or `VectorStoreRetrieverMemory`.
-   Combine memory with other Langchain components like Agents and Tools.
-   Experiment with `return_messages=True` in memory classes to get structured message objects instead of a formatted string.
-   Investigate how to persist and load memory for conversations that span multiple sessions.

Happy building with Langchain!